# Premios Nobel
### EDA básico con PySpark que consume la API de los Premios Nobel.

Instalamos `pyspark` en nuestra Notebook, usamos la flag `-q` o `--quiet` para evitar ver los mensajes relacionados con la descarga e instalación de la herramienta.

In [ ]:
!pip install -q pyspark

Creamos una SparkSession con el nombre del proyecto.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PremiosNobel") \
    .getOrCreate()

Usamos `requests` para leer la API de los Premios Nobel v1 y añadimos un manejo de errores básico, no es necesario instalarlo con `pip`.

In [ ]:
import requests

url = "https://api.nobelprize.org/v1/laureate.json"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    raise Exception(f"Error al obtener datos: {response.status_code}")

Creamos y visualizamos un DataFrame de Spark.

In [ ]:
laureates_df = spark.createDataFrame(data["laureates"])
laureates_df.show()

+----------+--------------------+--------------------+---------------+----------+------------+--------------------+---------------+----------------+------+---+--------------------+----------------+
|      born|            bornCity|         bornCountry|bornCountryCode|      died|    diedCity|         diedCountry|diedCountryCode|       firstname|gender| id|              prizes|         surname|
+----------+--------------------+--------------------+---------------+----------+------------+--------------------+---------------+----------------+------+---+--------------------+----------------+
|1845-03-27|Lennep (now Remsc...|Prussia (now Germ...|             DE|1923-02-10|      Munich|             Germany|             DE|  Wilhelm Conrad|  male|  1|[{affiliations ->...|         Röntgen|
|1853-07-18|              Arnhem|     the Netherlands|             NL|1928-02-04|        NULL|     the Netherlands|             NL|      Hendrik A.|  male|  2|[{affiliations ->...|         Lorentz|
|1865-05-2

Filtramos sólo las laureadas femeninas.

In [ ]:
females_df = laureates_df.filter(laureates_df.gender == "female")
females_df.show()

+----------+--------------------+--------------------+---------------+----------+-----------------+----------------+---------------+----------------+------+---+--------------------+----------------+
|      born|            bornCity|         bornCountry|bornCountryCode|      died|         diedCity|     diedCountry|diedCountryCode|       firstname|gender| id|              prizes|         surname|
+----------+--------------------+--------------------+---------------+----------+-----------------+----------------+---------------+----------------+------+---+--------------------+----------------+
|1867-11-07|              Warsaw|Russian Empire (n...|             PL|1934-07-04|       Sallanches|          France|             FR|           Marie|female|  6|[{affiliations ->...|           Curie|
|1906-06-28|Kattowitz (now Ka...|Germany (now Poland)|             PL|1972-02-20|    San Diego, CA|             USA|             US|           Maria|female| 79|[{affiliations ->...|  Goeppert Mayer|
|1897

Imprimimos el esquema de datos, cantidad de filas y columnas.

In [ ]:
females_df.printSchema()
print(f"Filas: {females_df.count()}, Columnas: {len(females_df.columns)}")

root
 |-- born: string (nullable = true)
 |-- bornCity: string (nullable = true)
 |-- bornCountry: string (nullable = true)
 |-- bornCountryCode: string (nullable = true)
 |-- died: string (nullable = true)
 |-- diedCity: string (nullable = true)
 |-- diedCountry: string (nullable = true)
 |-- diedCountryCode: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- id: string (nullable = true)
 |-- prizes: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- surname: string (nullable = true)

Filas: 65, Columnas: 13


Resumen estadístico de los datos.

In [ ]:
females_df.describe().show()

+-------+----------+---------------+-----------+---------------+----------+--------+--------------+---------------+---------+------+-----------------+-----------+
|summary|      born|       bornCity|bornCountry|bornCountryCode|      died|diedCity|   diedCountry|diedCountryCode|firstname|gender|               id|    surname|
+-------+----------+---------------+-----------+---------------+----------+--------+--------------+---------------+---------+------+-----------------+-----------+
|  count|        65|             65|         65|             65|        65|      29|            29|             29|       65|    65|               65|         64|
|   mean|      NULL|           NULL|       NULL|           NULL|      NULL|    NULL|          NULL|           NULL|     NULL|  NULL| 724.723076923077|       NULL|
| stddev|      NULL|           NULL|       NULL|           NULL|      NULL|    NULL|          NULL|           NULL|     NULL|  NULL|254.8234994371293|       NULL|
|    min|1843-06-09|  

Hacemos un `distinct` por país de nacimiento de las premiadas.

In [ ]:
females_df.select("bornCountry").distinct().show()

+--------------------+
|         bornCountry|
+--------------------+
|Austria-Hungary (...|
|Russian Empire (n...|
|             Germany|
|              France|
|Austrian Empire (...|
|               Italy|
|Germany (now Poland)|
|                 USA|
|               Egypt|
|               Yemen|
|              Sweden|
|         Philippines|
|                Iraq|
| Burma (now Myanmar)|
|               China|
|               Chile|
|              Norway|
|             Denmark|
|                Iran|
|             Ukraine|
+--------------------+
only showing top 20 rows



Filtramos por un país de nacimiento y seleccionamos las columnas a visualizar.

In [ ]:
filtered_df = females_df.select("bornCountry", "firstname", "surname").filter(females_df.bornCountry == "Chile")
filtered_df.show()

+-----------+---------+-------+
|bornCountry|firstname|surname|
+-----------+---------+-------+
|      Chile| Gabriela|Mistral|
+-----------+---------+-------+



Utilizamos las `udf` de Spark para convertir el formato de una columna a mayúsculas.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

uppercase_udf = udf(lambda x: x.upper() if x is not None else None, StringType())
df_upper = females_df.select("surname", "firstname").withColumn("upper", uppercase_udf("firstname"))
df_upper.show()

+----------------+----------------+----------------+
|         surname|       firstname|           upper|
+----------------+----------------+----------------+
|           Curie|           Marie|           MARIE|
|  Goeppert Mayer|           Maria|           MARIA|
|    Joliot-Curie|           Irène|           IRÈNE|
|         Hodgkin|Dorothy Crowfoot|DOROTHY CROWFOOT|
|            Cori|           Gerty|           GERTY|
|           Yalow|         Rosalyn|         ROSALYN|
|      McClintock|         Barbara|         BARBARA|
| Levi-Montalcini|            Rita|            RITA|
|           Elion|     Gertrude B.|     GERTRUDE B.|
|Nüsslein-Volhard|      Christiane|      CHRISTIANE|
|     von Suttner|          Bertha|          BERTHA|
|          Addams|            Jane|            JANE|
|           Balch|    Emily Greene|    EMILY GREENE|
|        Williams|           Betty|           BETTY|
|        Corrigan|         Mairead|         MAIREAD|
|        Bojaxhiu|   Anjezë Gonxhe|   ANJEZË G

Creamos una vista temportal con Spark, también utilizamos la MAGIC %%time que nos provee Google Colab para medir el impacto de recursos al ejecutar la celda.

In [ ]:
%%time
females_df.createOrReplaceTempView("females_vw")

+--------------------+
|         bornCountry|
+--------------------+
|Austria-Hungary (...|
|Russian Empire (n...|
|             Germany|
|              France|
|Austrian Empire (...|
|               Italy|
|Germany (now Poland)|
|                 USA|
|               Egypt|
|               Yemen|
|              Sweden|
|         Philippines|
|                Iraq|
| Burma (now Myanmar)|
|               China|
|               Chile|
|              Norway|
|             Denmark|
|                Iran|
|             Ukraine|
+--------------------+
only showing top 20 rows

CPU times: user 11.5 ms, sys: 937 µs, total: 12.4 ms
Wall time: 1.03 s


Realizamos la lectura de la vista temporal mediante lenguaje `sql`.

In [ ]:
spark.sql("SELECT DISTINCT bornCountry FROM females_vw").show()

Podríamos hacer lo siguiente, pero solo funciona en Databricks o donde se pueda utilizar %sql como una MAGIC. Actualmente Google Colab y Jupyter no lo permiten.

In [ ]:
%sql
SELECT DISTINCT bornCountry FROM females_vw

SyntaxError: invalid syntax (<ipython-input-22-358191dca53b>, line 2)